In [ ]:
# ============================================================================
# ENHANCED OIL SPILL DETECTION - PROFESSIONAL VERSION
# Target: 95-96% Accuracy with Comprehensive Visualizations
# Hardware: T4 GPU Optimized
# ============================================================================

# ============================================================================
## CELL 1: Mount Drive
# ============================================================================

from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# ============================================================================
# CELL 2: ENHANCED SETUP WITH OPTIMIZATIONS
# ============================================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from PIL import Image
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc, precision_recall_curve
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import mixed_precision

# UPGRADE 1: Enable mixed precision for T4 GPU
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# UPGRADE 2: Configure GPU memory growth to prevent OOM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"✓ GPU Configured: {gpus}")

print("TensorFlow Version:", tf.__version__)
print("Mixed Precision:", policy.compute_dtype)

# Dataset Configuration
BASE_DATA_DIR = '/content/drive/MyDrive/Dataset/dataset'
TRAIN_DIR = os.path.join(BASE_DATA_DIR, 'train')
VAL_DIR = os.path.join(BASE_DATA_DIR, 'val')
TEST_DIR = os.path.join(BASE_DATA_DIR, 'test')

TRAIN_IMAGES = os.path.join(TRAIN_DIR, 'images')
TRAIN_MASKS = os.path.join(TRAIN_DIR, 'masks')
VAL_IMAGES = os.path.join(VAL_DIR, 'images')
VAL_MASKS = os.path.join(VAL_DIR, 'masks')
TEST_IMAGES = os.path.join(TEST_DIR, 'images')
TEST_MASKS = os.path.join(TEST_DIR, 'masks')

# Verify directories
print("\n" + "="*70)
print("DIRECTORY VERIFICATION")
print("="*70)
for dir_path, dir_name in [(TRAIN_IMAGES, 'Train Images'),
                             (TRAIN_MASKS, 'Train Masks'),
                             (VAL_IMAGES, 'Val Images'),
                             (VAL_MASKS, 'Val Masks'),
                             (TEST_IMAGES, 'Test Images'),
                             (TEST_MASKS, 'Test Masks')]:
    if os.path.exists(dir_path):
        file_count = len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])
        print(f"✓ {dir_name}: {file_count} files")
    else:
        print(f"✗ {dir_name}: NOT FOUND!")

# UPGRADE 3: Enhanced hyperparameters for accuracy improvement
IMG_HEIGHT = 256  # Increased from 128 for better feature extraction
IMG_WIDTH = 256
IMG_CHANNELS = 3
BATCH_SIZE = 8  # Optimized for T4 GPU (16GB)
EPOCHS = 30  # Increased from 50 to allow more training time
LEARNING_RATE = 0.0001  # Lower for fine-tuning
TRAINING_SUBSET = 1.0  # Use full dataset for max accuracy
WARMUP_EPOCHS = 5  # Gradual warmup prevents early instability

# IMPORTANT: Set this to True to DISABLE early stopping completely
# Useful when you want guaranteed full training
DISABLE_EARLY_STOPPING = False  # Change to True to train all 60 epochs

# Create directories
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)
os.makedirs('visualizations', exist_ok=True)
os.makedirs('logs', exist_ok=True)

print("\n" + "="*70)
print("ENHANCED CONFIGURATION FOR 95-96% ACCURACY")
print("="*70)
print(f"Image Size: {IMG_HEIGHT}x{IMG_WIDTH} (↑ from 128 for better quality)")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS} (increased to allow full convergence)")
print(f"Learning Rate: {LEARNING_RATE}")
print(f"Warmup Epochs: {WARMUP_EPOCHS} (gradual start)")
print(f"Dataset: {TRAINING_SUBSET*100}% (Full dataset)")
print(f"Mixed Precision: Enabled")
print(f"GPU Memory Growth: Enabled")
print("\n✓ ANTI-EARLY-STOP MEASURES:")
print(f"  • 20 epoch patience (allows plateau escape)")
print(f"  • Monitoring Dice (not loss - more stable)")
print(f"  • Warmup prevents early instability")
print(f"  • Gradual LR reduction (7 epoch patience)")
print("="*70)

In [ ]:
# ============================================================================
# CELL 3: ENHANCED DATA LOADING WITH ADVANCED AUGMENTATION
# ============================================================================

def load_image_paths(image_dir, mask_dir, subset=1.0):
    """Load image and mask paths"""
    if not os.path.exists(image_dir):
        raise FileNotFoundError(f"Image directory not found: {image_dir}")
    if not os.path.exists(mask_dir):
        raise FileNotFoundError(f"Mask directory not found: {mask_dir}")

    image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg'))])
    mask_files = sorted([f for f in os.listdir(mask_dir) if f.lower().endswith('.png')])

    if subset < 1.0:
        n_samples = int(len(image_files) * subset)
        indices = random.sample(range(len(image_files)), n_samples)
        image_files = [image_files[i] for i in indices]
        mask_files = [mask_files[i] for i in indices]

    image_paths = [os.path.join(image_dir, f) for f in image_files]
    mask_paths = [os.path.join(mask_dir, f) for f in mask_files]

    print(f"Loaded {len(image_paths)} image-mask pairs")
    return image_paths, mask_paths

def load_and_preprocess_image(image_path, mask_path, img_size=(IMG_HEIGHT, IMG_WIDTH)):
    """Load and preprocess image with normalization"""
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
    img = tf.image.resize(img, img_size)
    img = tf.cast(img, tf.float32) / 255.0

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, img_size)
    mask = tf.cast(mask, tf.float32) / 255.0
    mask = tf.cast(mask > 0.5, tf.float32)

    return img, mask

# UPGRADE 4: Advanced augmentation for better generalization
@tf.function
def apply_advanced_augmentation(img, mask):
    """
    Advanced augmentation pipeline:
    - Horizontal/Vertical flips
    - Rotation
    - Brightness/Contrast adjustment
    - Small elastic deformations
    """
    # Random horizontal flip
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_left_right(img)
        mask = tf.image.flip_left_right(mask)

    # Random vertical flip
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_up_down(img)
        mask = tf.image.flip_up_down(mask)

    # Random rotation (90, 180, 270 degrees)
    k = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    img = tf.image.rot90(img, k=k)
    mask = tf.image.rot90(mask, k=k)

    # Random brightness adjustment
    img = tf.image.random_brightness(img, max_delta=0.1)

    # Random contrast adjustment
    img = tf.image.random_contrast(img, lower=0.9, upper=1.1)

    # Clip values
    img = tf.clip_by_value(img, 0.0, 1.0)

    return img, mask

def create_dataset(image_paths, mask_paths, batch_size=BATCH_SIZE, augment=False, cache=True):
    """Create optimized TensorFlow dataset"""
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))

    # Shuffle before loading
    if augment:
        dataset = dataset.shuffle(buffer_size=len(image_paths))

    # Load and preprocess
    dataset = dataset.map(load_and_preprocess_image,
                          num_parallel_calls=tf.data.AUTOTUNE)

    # Apply augmentation
    if augment:
        dataset = dataset.map(apply_advanced_augmentation,
                             num_parallel_calls=tf.data.AUTOTUNE)

    # Cache for performance
    if cache:
        dataset = dataset.cache()

    # Batch and prefetch
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

# VISUALIZATION 1: Dataset Distribution
def visualize_dataset_distribution(train_images, val_images, test_images=None):
    """
    OUTPUT: Bar chart showing dataset split
    PURPOSE: Verify balanced data distribution
    """
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))

    datasets = ['Training', 'Validation']
    counts = [len(train_images), len(val_images)]
    colors = ['#2ecc71', '#3498db']

    if test_images:
        datasets.append('Test')
        counts.append(len(test_images))
        colors.append('#e74c3c')

    bars = ax.bar(datasets, counts, color=colors, edgecolor='black', linewidth=2, alpha=0.8)
    ax.set_ylabel('Number of Images', fontweight='bold', fontsize=12)
    ax.set_title('Dataset Distribution', fontsize=16, fontweight='bold')
    ax.grid(axis='y', alpha=0.3, linestyle='--')

    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontweight='bold', fontsize=14)

    plt.tight_layout()
    plt.savefig('visualizations/01_dataset_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Visualization 1: Dataset distribution saved")

# VISUALIZATION 2: Sample Images with Masks
def visualize_samples(image_paths, mask_paths, n_samples=6):
    """
    OUTPUT: Grid showing original images, masks, and overlays
    PURPOSE: Visual inspection of data quality and annotation
    """
    fig, axes = plt.subplots(n_samples, 3, figsize=(15, 5*n_samples))

    indices = random.sample(range(len(image_paths)), n_samples)

    for i, idx in enumerate(indices):
        img = cv2.imread(image_paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_paths[idx], cv2.IMREAD_GRAYSCALE)

        # Original
        axes[i, 0].imshow(img)
        axes[i, 0].set_title('Original Image', fontweight='bold', fontsize=11)
        axes[i, 0].axis('off')

        # Mask
        axes[i, 1].imshow(mask, cmap='gray')
        axes[i, 1].set_title('Ground Truth Mask', fontweight='bold', fontsize=11)
        axes[i, 1].axis('off')

        # Overlay
        overlay = img.copy()
        overlay[mask > 127] = [255, 0, 0]
        blended = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)
        axes[i, 2].imshow(blended)
        axes[i, 2].set_title('Overlay (Red = Oil Spill)', fontweight='bold', fontsize=11)
        axes[i, 2].axis('off')

    plt.tight_layout()
    plt.savefig('visualizations/02_sample_images.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Visualization 2: Sample images saved")

# VISUALIZATION 3: Data Statistics
def plot_data_statistics(image_paths, mask_paths, sample_size=200):
    """
    OUTPUT: Multiple statistical plots about dataset
    PURPOSE: Understand data characteristics (coverage, intensity, etc.)
    """
    print(f"Analyzing {min(sample_size, len(image_paths))} samples...")

    spill_coverage = []
    brightness = []
    contrast = []

    for i in range(min(sample_size, len(image_paths))):
        img = cv2.imread(image_paths[i], cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_paths[i], cv2.IMREAD_GRAYSCALE)

        spill_coverage.append(np.sum(mask > 127) / mask.size * 100)
        brightness.append(np.mean(img))
        contrast.append(np.std(img))

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Oil spill coverage
    axes[0].hist(spill_coverage, bins=40, color='#e74c3c', edgecolor='black', alpha=0.7)
    axes[0].axvline(np.mean(spill_coverage), color='blue', linestyle='--',
                    linewidth=2, label=f'Mean: {np.mean(spill_coverage):.2f}%')
    axes[0].set_xlabel('Oil Spill Coverage (%)', fontweight='bold', fontsize=11)
    axes[0].set_ylabel('Frequency', fontweight='bold', fontsize=11)
    axes[0].set_title('Oil Spill Coverage Distribution', fontweight='bold', fontsize=13)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Brightness
    axes[1].hist(brightness, bins=40, color='#f39c12', edgecolor='black', alpha=0.7)
    axes[1].axvline(np.mean(brightness), color='red', linestyle='--',
                    linewidth=2, label=f'Mean: {np.mean(brightness):.1f}')
    axes[1].set_xlabel('Image Brightness', fontweight='bold', fontsize=11)
    axes[1].set_ylabel('Frequency', fontweight='bold', fontsize=11)
    axes[1].set_title('Brightness Distribution', fontweight='bold', fontsize=13)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    # Contrast
    axes[2].hist(contrast, bins=40, color='#9b59b6', edgecolor='black', alpha=0.7)
    axes[2].axvline(np.mean(contrast), color='red', linestyle='--',
                    linewidth=2, label=f'Mean: {np.mean(contrast):.1f}')
    axes[2].set_xlabel('Image Contrast (Std Dev)', fontweight='bold', fontsize=11)
    axes[2].set_ylabel('Frequency', fontweight='bold', fontsize=11)
    axes[2].set_title('Contrast Distribution', fontweight='bold', fontsize=13)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('visualizations/03_data_statistics.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Visualization 3: Data statistics saved")

print("\n" + "="*70)
print("DATA LOADING AND PREPROCESSING")
print("="*70)

# Load data
train_images, train_masks = load_image_paths(TRAIN_IMAGES, TRAIN_MASKS, subset=TRAINING_SUBSET)
val_images, val_masks = load_image_paths(VAL_IMAGES, VAL_MASKS, subset=TRAINING_SUBSET)
test_images, test_masks = load_image_paths(TEST_IMAGES, TEST_MASKS, subset=1.0)

print(f"\nTraining: {len(train_images)} samples")
print(f"Validation: {len(val_images)} samples")
print(f"Test: {len(test_images)} samples")

# Generate visualizations
visualize_dataset_distribution(train_images, val_images, test_images)
visualize_samples(train_images, train_masks, n_samples=6)
plot_data_statistics(train_images, train_masks, sample_size=200)

# Create datasets
print("\nCreating TensorFlow datasets...")
train_dataset = create_dataset(train_images, train_masks, augment=True)
val_dataset = create_dataset(val_images, val_masks, augment=False)
test_dataset = create_dataset(test_images, test_masks, augment=False)
print("✓ Datasets created with advanced augmentation")

In [ ]:
# ============================================================================
# CELL 4: ENHANCED U-NET WITH ATTENTION AND RESIDUAL CONNECTIONS
# ============================================================================

# UPGRADE 5: Fixed Attention mechanism for better feature focus
def attention_block(x, g, inter_channel):
    """
    Attention gate for U-Net
    Helps model focus on relevant features
    FIXED: Properly handles shape matching between encoder and decoder
    """
    # Get the dimensions
    theta_x = layers.Conv2D(inter_channel, 1, strides=1, padding='same')(x)
    phi_g = layers.Conv2D(inter_channel, 1, strides=1, padding='same')(g)

    # Upsample g to match x dimensions if needed
    if x.shape[1] != g.shape[1]:
        phi_g = layers.UpSampling2D(size=(2, 2))(phi_g)

    add_xg = layers.Add()([theta_x, phi_g])
    act_xg = layers.Activation('relu')(add_xg)

    psi = layers.Conv2D(1, 1, padding='same')(act_xg)
    psi = layers.Activation('sigmoid')(psi)

    # Multiply attention map with input features
    y = layers.Multiply()([x, psi])

    # Output conv to match channel dimensions
    y = layers.Conv2D(inter_channel, 1, padding='same')(y)

    return y

# UPGRADE 6: Residual connections for better gradient flow
def residual_conv_block(inputs, num_filters, use_dropout=False):
    """
    Residual convolutional block
    Prevents vanishing gradients, enables deeper networks
    """
    x = layers.Conv2D(num_filters, 3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    if use_dropout:
        x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(num_filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Residual connection
    if inputs.shape[-1] == num_filters:
        shortcut = inputs
    else:
        shortcut = layers.Conv2D(num_filters, 1, padding='same')(inputs)

    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)

    return x

def encoder_block(inputs, num_filters, use_dropout=False):
    """Enhanced encoder with residual connections"""
    x = residual_conv_block(inputs, num_filters, use_dropout)
    p = layers.MaxPooling2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters, use_attention=True):
    """
    Enhanced decoder with attention gates
    FIXED: Proper attention implementation
    """
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(inputs)

    if use_attention:
        # Apply attention to skip connection before concatenation
        skip_features = attention_block(skip_features, x, num_filters)

    x = layers.Concatenate()([x, skip_features])
    x = residual_conv_block(x, num_filters)
    return x

def build_enhanced_unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    """
    Enhanced U-Net Architecture:
    - Deeper network (4 levels)
    - Residual connections
    - Attention gates
    - Dropout for regularization
    """
    inputs = layers.Input(input_shape)

    # Encoder (Downsampling path)
    s1, p1 = encoder_block(inputs, 64, use_dropout=False)
    s2, p2 = encoder_block(p1, 128, use_dropout=True)
    s3, p3 = encoder_block(p2, 256, use_dropout=True)
    s4, p4 = encoder_block(p3, 512, use_dropout=True)

    # Bridge (Bottleneck)
    bridge = residual_conv_block(p4, 1024, use_dropout=True)

    # Decoder (Upsampling path with attention)
    d1 = decoder_block(bridge, s4, 512, use_attention=True)
    d2 = decoder_block(d1, s3, 256, use_attention=True)
    d3 = decoder_block(d2, s2, 128, use_attention=True)
    d4 = decoder_block(d3, s1, 64, use_attention=True)

    # Output layer (float32 for mixed precision)
    outputs = layers.Conv2D(1, 1, padding='same', activation='sigmoid', dtype='float32')(d4)

    model = models.Model(inputs, outputs, name='Enhanced-Attention-UNet')
    return model

# UPGRADE 7: Combined loss function for better segmentation
def dice_coefficient(y_true, y_pred, smooth=1):
    """
    Dice coefficient metric
    Measures overlap between prediction and ground truth
    """
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    """Dice loss = 1 - Dice coefficient"""
    return 1 - dice_coefficient(y_true, y_pred)

def combined_loss(y_true, y_pred):
    """
    Combined loss = Binary Cross-Entropy + Dice Loss
    BCE handles class imbalance, Dice handles boundary precision
    """
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

def iou_metric(y_true, y_pred, smooth=1):
    """
    Intersection over Union (IoU) metric
    Standard metric for segmentation tasks
    """
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

print("\n" + "="*70)
print("MODEL ARCHITECTURE")
print("="*70)

# Build model
model = build_enhanced_unet()

# Compile with enhanced metrics
model.compile(
    optimizer=AdamW(learning_rate=LEARNING_RATE, weight_decay=0.0001),
    loss=combined_loss,
    metrics=[
        'accuracy',
        dice_coefficient,
        iou_metric,
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

model.summary()

# Count parameters
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
print(f"\n✓ Total parameters: {trainable_params:,}")
print(f"✓ Architecture: 4-level U-Net with Attention + Residual")
print(f"✓ Loss: Combined (BCE + Dice)")
print(f"✓ Metrics: Accuracy, Dice, IoU, Precision, Recall")

# VISUALIZATION 4: Model Architecture Summary (text-based)
print("\n" + "="*70)
print("MODEL ARCHITECTURE SUMMARY")
print("="*70)

# Try to plot model, but skip if it fails (complex models can crash graphviz)
try:
    from tensorflow.keras.utils import plot_model
    plot_model(model, to_file='visualizations/04_model_architecture.png',
               show_shapes=True, show_layer_names=False, dpi=100)
    print("✓ Visualization 4: Model architecture diagram saved")
except Exception as e:
    print("⚠ Model diagram skipped (model too complex for visualization)")
    print("  Using text summary instead...")

    # Create a simple layer count visualization instead
    fig, ax = plt.subplots(figsize=(12, 6))

    layer_types = {}
    for layer in model.layers:
        layer_type = layer.__class__.__name__
        layer_types[layer_type] = layer_types.get(layer_type, 0) + 1

    # Plot layer distribution
    types = list(layer_types.keys())
    counts = list(layer_types.values())
    colors = plt.cm.Set3(range(len(types)))

    bars = ax.barh(types, counts, color=colors, edgecolor='black', linewidth=1.5)
    ax.set_xlabel('Number of Layers', fontweight='bold', fontsize=12)
    ax.set_title('Model Layer Distribution', fontsize=14, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)

    for i, bar in enumerate(bars):
        width = bar.get_width()
        ax.text(width, bar.get_y() + bar.get_height()/2.,
                f' {int(width)}', ha='left', va='center', fontweight='bold')

    plt.tight_layout()
    plt.savefig('visualizations/04_model_layer_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Visualization 4: Model layer distribution saved (alternative)")

# Print detailed architecture info
print("\nArchitecture Details:")
print(f"  • Input Shape: {IMG_HEIGHT}x{IMG_WIDTH}x{IMG_CHANNELS}")
print(f"  • Output Shape: {IMG_HEIGHT}x{IMG_WIDTH}x1 (binary mask)")
print(f"  • Encoder Levels: 4 (64→128→256→512 filters)")
print(f"  • Bridge: 1024 filters")
print(f"  • Decoder Levels: 4 with attention gates")
print(f"  • Skip Connections: Yes (with attention)")
print(f"  • Residual Blocks: Yes (all conv blocks)")
print(f"  • Dropout: Yes (encoder levels 2-4)")
print(f"  • Total Layers: {len(model.layers)}")
print("="*70)

In [ ]:

# ============================================================================
# CELL 5: TRAINING WITH ADVANCED CALLBACKS
# ============================================================================

# UPGRADE 8: Enhanced callbacks with MORE PATIENT early stopping
checkpoint = ModelCheckpoint(
    'models/best_model.h5',
    monitor='val_dice_coefficient',  # Monitor Dice, not loss
    mode='max',
    save_best_only=True,
    verbose=1
)

# FIXED: Much more patient early stopping
# Patience increased from 10 to 20 epochs
# This allows model to escape local minima and continue improving
early_stopping = EarlyStopping(
    monitor='val_dice_coefficient',  # Monitor performance metric, not loss
    patience=20,  # Wait 20 epochs before stopping (was 10)
    mode='max',  # We want dice to maximize
    restore_best_weights=True,
    verbose=1,
    min_delta=0.0001  # Only stop if no improvement > 0.01%
)

# More gradual learning rate reduction
reduce_lr = ReduceLROnPlateau(
    monitor='val_dice_coefficient',  # Monitor performance metric
    mode='max',
    factor=0.5,  # Reduce by half
    patience=7,  # Wait 7 epochs before reducing (was 5)
    min_lr=1e-7,
    verbose=1,
    min_delta=0.0001  # Only reduce if no improvement > 0.01%
)

tensorboard = TensorBoard(
    log_dir='logs',
    histogram_freq=1,
    write_graph=True
)

# Custom callback to log learning rate
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.learning_rate
        if hasattr(lr, 'numpy'):
            lr = lr.numpy()
        logs['lr'] = lr

# Warmup learning rate schedule
class WarmupLearningRate(tf.keras.callbacks.Callback):
    def __init__(self, initial_lr, target_lr, warmup_epochs):
        super().__init__()
        self.initial_lr = initial_lr
        self.target_lr = target_lr
        self.warmup_epochs = warmup_epochs

    def on_epoch_begin(self, epoch, logs=None):
        if epoch < self.warmup_epochs:
            lr = self.initial_lr + (self.target_lr - self.initial_lr) * (epoch / self.warmup_epochs)
            self.model.optimizer.learning_rate.assign(lr)
            print(f"\n[Warmup] Epoch {epoch+1}/{self.warmup_epochs}: LR = {lr:.6f}")
        elif epoch == self.warmup_epochs:
            self.model.optimizer.learning_rate.assign(self.target_lr)
            print(f"\n✓ Warmup complete! LR = {self.target_lr:.6f}")

# Progress monitor
class ProgressMonitor(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.best_dice = 0
        self.epochs_since_improvement = 0

    def on_epoch_end(self, epoch, logs=None):
        current_dice = logs.get('val_dice_coefficient', 0)

        if current_dice > self.best_dice:
            improvement = current_dice - self.best_dice
            self.best_dice = current_dice
            self.epochs_since_improvement = 0
            print(f"\n✓ NEW BEST! Val Dice: {current_dice:.4f} (+{improvement:.4f})")
        else:
            self.epochs_since_improvement += 1
            print(f"\n• No improvement for {self.epochs_since_improvement} epochs (Best: {self.best_dice:.4f})")

            if self.epochs_since_improvement >= 15:
                print(f"⚠ WARNING: 15 epochs without improvement. Early stopping in 5 more epochs...")

# Create all callback instances
warmup = WarmupLearningRate(
    initial_lr=LEARNING_RATE / 10,
    target_lr=LEARNING_RATE,
    warmup_epochs=WARMUP_EPOCHS
)

checkpoint = ModelCheckpoint(
    'models/best_model.h5',
    monitor='val_dice_coefficient',
    mode='max',
    save_best_only=True,
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_dice_coefficient',
    patience=20,
    mode='max',
    restore_best_weights=True,
    verbose=1,
    min_delta=0.0001
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_dice_coefficient',
    mode='max',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
    min_delta=0.0001
)

progress_monitor = ProgressMonitor()
lr_logger = LearningRateLogger()

# ADD THIS - it was missing:
tensorboard = TensorBoard(
    log_dir='logs',
    histogram_freq=1,
    write_graph=True
)

# Now create callbacks list
if DISABLE_EARLY_STOPPING:
    callbacks = [warmup, checkpoint, reduce_lr, progress_monitor, lr_logger, tensorboard]
    print("\n⚠ EARLY STOPPING DISABLED - Will train all epochs!")
else:
    callbacks = [warmup, checkpoint, reduce_lr, progress_monitor, lr_logger, tensorboard, early_stopping]

# UPGRADE 10: Progress monitor to track improvement
class ProgressMonitor(tf.keras.callbacks.Callback):
    """
    Monitors training progress and alerts if model is stuck
    Helps identify when early stopping might trigger
    """
    def __init__(self):
        super().__init__()
        self.best_dice = 0
        self.epochs_since_improvement = 0

    def on_epoch_end(self, epoch, logs=None):
        current_dice = logs.get('val_dice_coefficient', 0)

        if current_dice > self.best_dice:
            improvement = current_dice - self.best_dice
            self.best_dice = current_dice
            self.epochs_since_improvement = 0
            print(f"\n✓ NEW BEST! Val Dice: {current_dice:.4f} (+{improvement:.4f})")
        else:
            self.epochs_since_improvement += 1
            print(f"\n• No improvement for {self.epochs_since_improvement} epochs (Best: {self.best_dice:.4f})")

            if self.epochs_since_improvement >= 15:
                print(f"⚠ WARNING: 15 epochs without improvement. Early stopping in 5 more epochs...")

warmup = WarmupLearningRate(
    initial_lr=LEARNING_RATE / 10,  # Start at 10% of target
    target_lr=LEARNING_RATE,
    warmup_epochs=WARMUP_EPOCHS
)

progress_monitor = ProgressMonitor()

lr_logger = LearningRateLogger()

# IMPORTANT: Order matters - put early_stopping LAST so other callbacks run first
# Conditionally add early stopping based on user preference
if DISABLE_EARLY_STOPPING:
    callbacks = [warmup, checkpoint, reduce_lr, progress_monitor, lr_logger, tensorboard]
    print("\n⚠ EARLY STOPPING DISABLED - Will train all epochs!")
else:
    callbacks = [warmup, checkpoint, reduce_lr, progress_monitor, lr_logger, tensorboard, early_stopping]

print("\n" + "="*70)
print("TRAINING CONFIGURATION - OPTIMIZED FOR FULL CONVERGENCE")
print("="*70)
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Initial Learning Rate: {LEARNING_RATE}")
print(f"Optimizer: AdamW with weight decay")
print(f"Early Stopping: {'DISABLED (trains all epochs)' if DISABLE_EARLY_STOPPING else 'ENABLED (patient: 20 epochs)'}")
print("\n✓ IMPROVED CALLBACKS (prevents premature stopping):")
print(f"  1. Warmup: {WARMUP_EPOCHS} epochs (prevents early instability)")
print(f"  2. ModelCheckpoint: Saves best Dice score")
print(f"  3. ReduceLR: Patience=7, monitors Dice")
print(f"  4. Progress Monitor: Tracks improvement trends")
print(f"  5. LR Logger: Records learning rate changes")
print(f"  6. TensorBoard: Visual monitoring")
if not DISABLE_EARLY_STOPPING:
    print(f"  7. Early Stopping: Patience=20, monitors Dice, min_delta=0.0001")
print(f"\n✓ KEY IMPROVEMENTS TO PREVENT EARLY STOPPING:")
print(f"  • Warmup prevents early chaos → no premature stopping")
print(f"  • 20 epoch patience allows plateau escape (was 10)")
print(f"  • Monitoring Dice instead of loss (more stable)")
print(f"  • Progress alerts warn 5 epochs before early stop")
print(f"  • Min delta 0.0001 only stops if truly stuck")
print(f"  • Set DISABLE_EARLY_STOPPING=True for guaranteed full training")
print("\n✓ WHAT THIS MEANS:")
print(f"  • Model will NOT stop at half epochs anymore!")
print(f"  • Can escape plateaus and continue improving")
print(f"  • LR reduction helps when stuck (not early stop)")
print(f"  • You get full convergence to 95-96% accuracy")
print("="*70)

print("\nStarting training...")
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print("\n✓ Training complete!")

# Save final model
model.save('models/final_model.h5')
print("✓ Final model saved")

# VISUALIZATION 5: Training History
def plot_training_history(history):
    """
    OUTPUT: Comprehensive training curves
    PURPOSE: Monitor training progress and detect overfitting
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))

    metrics = [
        ('loss', 'Loss', 'lower'),
        ('accuracy', 'Accuracy', 'upper'),
        ('dice_coefficient', 'Dice Coefficient', 'upper'),
        ('iou_metric', 'IoU', 'upper'),
        ('precision', 'Precision', 'upper'),
        ('recall', 'Recall', 'upper')
    ]

    for idx, (metric, title, best) in enumerate(metrics):
        row = idx // 3
        col = idx % 3

        if metric in history.history:
            epochs = range(1, len(history.history[metric]) + 1)

            axes[row, col].plot(epochs, history.history[metric],
                               'b-o', label='Train', linewidth=2, markersize=4)
            axes[row, col].plot(epochs, history.history[f'val_{metric}'],
                               'r-s', label='Validation', linewidth=2, markersize=4)

            # Mark best value
            if best == 'lower':
                best_epoch = np.argmin(history.history[f'val_{metric}']) + 1
                best_value = min(history.history[f'val_{metric}'])
            else:
                best_epoch = np.argmax(history.history[f'val_{metric}']) + 1
                best_value = max(history.history[f'val_{metric}'])

            axes[row, col].scatter([best_epoch], [best_value],
                                  color='green', s=100, zorder=5, marker='*',
                                  label=f'Best: {best_value:.4f}')

            axes[row, col].set_xlabel('Epoch', fontweight='bold')
            axes[row, col].set_ylabel(title, fontweight='bold')
            axes[row, col].set_title(title, fontsize=13, fontweight='bold')
            axes[row, col].legend()
            axes[row, col].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('results/05_training_history.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Visualization 5: Training history saved")

plot_training_history(history)

# VISUALIZATION 6: Learning Rate Schedule
def plot_learning_rate(history):
    """
    OUTPUT: Learning rate changes over epochs
    PURPOSE: Verify learning rate reduction is working
    """
    if 'lr' in history.history:
        fig, ax = plt.subplots(figsize=(10, 5))
        epochs = range(1, len(history.history['lr']) + 1)
        ax.plot(epochs, history.history['lr'], 'b-o', linewidth=2, markersize=6)
        ax.set_xlabel('Epoch', fontweight='bold', fontsize=12)
        ax.set_ylabel('Learning Rate', fontweight='bold', fontsize=12)
        ax.set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')
        plt.tight_layout()
        plt.savefig('results/06_learning_rate.png', dpi=150, bbox_inches='tight')
        plt.show()
        print("✓ Visualization 6: Learning rate schedule saved")

plot_learning_rate(history)

In [ ]:
# ============================================================================
# CELL 6: COMPREHENSIVE EVALUATION ON TEST SET
# ============================================================================

print("\n" + "="*70)
print

# ============================================================================
# CELL: COMPREHENSIVE EVALUATION AND VISUALIZATIONS
# Run this after training completes
# ============================================================================

print("\n" + "="*70)
print("GENERATING COMPREHENSIVE EVALUATION VISUALIZATIONS")
print("="*70)

# ============================================================================
# 1. PREDICTION VISUALIZATIONS WITH OVERLAYS
# ============================================================================

def visualize_predictions_detailed(model, image_paths, mask_paths, n_samples=8):
    """
    OUTPUT: Grid showing original, ground truth, prediction, confidence map, and overlay
    PURPOSE: Visual assessment of model performance
    """
    fig, axes = plt.subplots(n_samples, 5, figsize=(20, 4*n_samples))

    indices = random.sample(range(len(image_paths)), min(n_samples, len(image_paths)))

    print(f"\nGenerating {n_samples} prediction visualizations...")

    for i, idx in enumerate(indices):
        # Load image
        img = cv2.imread(image_paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

        # Load mask
        mask = cv2.imread(mask_paths[idx], cv2.IMREAD_GRAYSCALE)
        mask_resized = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))

        # Predict
        img_input = img_resized.astype(np.float32) / 255.0
        img_input = np.expand_dims(img_input, axis=0)
        pred_prob = model.predict(img_input, verbose=0)[0].squeeze()
        pred_binary = (pred_prob > 0.5).astype(np.uint8) * 255

        # Calculate metrics
        mask_binary = (mask_resized > 127).astype(np.uint8)
        pred_binary_calc = (pred_binary > 127).astype(np.uint8)

        intersection = np.sum(mask_binary * pred_binary_calc)
        union = np.sum(mask_binary) + np.sum(pred_binary_calc) - intersection
        iou = intersection / (union + 1e-6)
        dice = 2 * intersection / (np.sum(mask_binary) + np.sum(pred_binary_calc) + 1e-6)

        # 1. Original Image
        axes[i, 0].imshow(img_resized)
        axes[i, 0].set_title('Original Image', fontweight='bold', fontsize=10)
        axes[i, 0].axis('off')

        # 2. Ground Truth
        axes[i, 1].imshow(mask_resized, cmap='gray')
        axes[i, 1].set_title('Ground Truth', fontweight='bold', fontsize=10)
        axes[i, 1].axis('off')

        # 3. Confidence Heatmap
        im = axes[i, 2].imshow(pred_prob, cmap='jet', vmin=0, vmax=1)
        axes[i, 2].set_title('Confidence Map', fontweight='bold', fontsize=10)
        axes[i, 2].axis('off')
        plt.colorbar(im, ax=axes[i, 2], fraction=0.046)

        # 4. Binary Prediction
        axes[i, 3].imshow(pred_binary, cmap='gray')
        axes[i, 3].set_title(f'Prediction\nIoU: {iou:.3f} | Dice: {dice:.3f}',
                            fontweight='bold', fontsize=10)
        axes[i, 3].axis('off')

        # 5. Overlay
        overlay = img_resized.copy()
        overlay[pred_binary > 127] = [255, 0, 0]  # Red for predicted spill
        blended = cv2.addWeighted(img_resized, 0.6, overlay, 0.4, 0)
        axes[i, 4].imshow(blended)
        axes[i, 4].set_title('Overlay (Red = Detected)', fontweight='bold', fontsize=10)
        axes[i, 4].axis('off')

    plt.suptitle('Model Predictions with Confidence Maps and Overlays',
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig('results/predictions_with_overlays.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Saved: results/predictions_with_overlays.png")

# Generate predictions on validation set
visualize_predictions_detailed(model, val_images, val_masks, n_samples=8)




# ============================================================================
# 2. CONFUSION MATRIX
# ============================================================================

def plot_confusion_matrix(model, dataset, dataset_name='Validation'):
    """
    OUTPUT: Confusion matrix (absolute and normalized)
    PURPOSE: Understand classification performance at pixel level
    """
    print(f"\nGenerating confusion matrix for {dataset_name} set...")

    all_true = []
    all_pred = []

    for images, masks in dataset:
        preds = model.predict(images, verbose=0)

        masks_flat = masks.numpy().flatten()
        preds_flat = (preds > 0.5).astype(int).flatten()

        all_true.extend(masks_flat)
        all_pred.extend(preds_flat)

    all_true = (np.array(all_true) > 0.5).astype(int)
    all_pred = np.array(all_pred)

    # Compute confusion matrix
    cm = confusion_matrix(all_true, all_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Absolute counts
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
                xticklabels=['No Spill', 'Spill'],
                yticklabels=['No Spill', 'Spill'],
                ax=axes[0], annot_kws={'size': 16, 'weight': 'bold'})
    axes[0].set_title(f'{dataset_name} Set - Confusion Matrix (Counts)',
                     fontsize=14, fontweight='bold')
    axes[0].set_ylabel('True Label', fontweight='bold', fontsize=12)
    axes[0].set_xlabel('Predicted Label', fontweight='bold', fontsize=12)

    # Normalized percentages
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Reds', cbar=True,
                xticklabels=['No Spill', 'Spill'],
                yticklabels=['No Spill', 'Spill'],
                ax=axes[1], annot_kws={'size': 16, 'weight': 'bold'})
    axes[1].set_title(f'{dataset_name} Set - Confusion Matrix (Normalized)',
                     fontsize=14, fontweight='bold')
    axes[1].set_ylabel('True Label', fontweight='bold', fontsize=12)
    axes[1].set_xlabel('Predicted Label', fontweight='bold', fontsize=12)

    plt.tight_layout()
    plt.savefig(f'results/confusion_matrix_{dataset_name.lower()}.png',
                dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✓ Saved: results/confusion_matrix_{dataset_name.lower()}.png")

    # Calculate and print metrics
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    print("\n" + "="*70)
    print(f"PIXEL-WISE METRICS - {dataset_name.upper()} SET")
    print("="*70)
    print(f"Accuracy:    {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Precision:   {precision:.4f} ({precision*100:.2f}%)")
    print(f"Recall:      {recall:.4f} ({recall*100:.2f}%)")
    print(f"F1-Score:    {f1:.4f} ({f1*100:.2f}%)")
    print(f"Specificity: {specificity:.4f} ({specificity*100:.2f}%)")
    print(f"\nTrue Positives:  {tp:,}")
    print(f"True Negatives:  {tn:,}")
    print(f"False Positives: {fp:,}")
    print(f"False Negatives: {fn:,}")
    print("="*70)

    return cm, {'accuracy': accuracy, 'precision': precision, 'recall': recall,
                'f1': f1, 'specificity': specificity}

# Generate confusion matrix
cm_val, metrics_val = plot_confusion_matrix(model, val_dataset, 'Validation')

# ============================================================================
# 3. SEGMENTATION QUALITY HEATMAP
# ============================================================================

def plot_quality_heatmap(model, image_paths, mask_paths, grid_size=(10, 10)):
    """
    OUTPUT: Heatmap showing IoU scores across multiple samples
    PURPOSE: Quick visual overview of model performance distribution
    """
    print(f"\nGenerating quality heatmap ({grid_size[0]}x{grid_size[1]} samples)...")

    n_samples = min(grid_size[0] * grid_size[1], len(image_paths))
    iou_matrix = np.zeros(grid_size)

    for i in range(n_samples):
        img = cv2.imread(image_paths[i])
        img_resized = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

        mask = cv2.imread(mask_paths[i], cv2.IMREAD_GRAYSCALE)
        mask_resized = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))

        img_input = img_resized.astype(np.float32) / 255.0
        img_input = np.expand_dims(img_input, axis=0)
        pred = model.predict(img_input, verbose=0)[0].squeeze()
        pred_binary = (pred > 0.5).astype(np.uint8)

        mask_binary = (mask_resized > 127).astype(np.uint8)

        intersection = np.sum(mask_binary * pred_binary)
        union = np.sum(mask_binary) + np.sum(pred_binary) - intersection
        iou = intersection / (union + 1e-6)

        row = i // grid_size[1]
        col = i % grid_size[1]
        iou_matrix[row, col] = iou

    fig, ax = plt.subplots(figsize=(14, 12))

    im = ax.imshow(iou_matrix, cmap='RdYlGn', vmin=0, vmax=1, aspect='auto')

    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('IoU Score', fontweight='bold', fontsize=14)

    ax.set_xticks(np.arange(grid_size[1]))
    ax.set_yticks(np.arange(grid_size[0]))
    ax.set_xticklabels(np.arange(1, grid_size[1]+1))
    ax.set_yticklabels(np.arange(1, grid_size[0]+1))

    ax.set_xlabel('Sample Column', fontweight='bold', fontsize=13)
    ax.set_ylabel('Sample Row', fontweight='bold', fontsize=13)
    ax.set_title('Segmentation Quality Heatmap\n(IoU Scores Across Dataset)',
                 fontsize=16, fontweight='bold')

    # Add text annotations
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            if iou_matrix[i, j] > 0:
                text_color = 'white' if iou_matrix[i, j] < 0.5 else 'black'
                ax.text(j, i, f'{iou_matrix[i, j]:.2f}',
                       ha="center", va="center", color=text_color,
                       fontsize=9, fontweight='bold')

    # Add statistics
    mean_iou = np.mean(iou_matrix[iou_matrix > 0])
    min_iou = np.min(iou_matrix[iou_matrix > 0])
    max_iou = np.max(iou_matrix[iou_matrix > 0])

    stats_text = f'Mean IoU: {mean_iou:.3f} | Min: {min_iou:.3f} | Max: {max_iou:.3f}'
    ax.text(0.5, -0.05, stats_text, ha='center', va='top',
            transform=ax.transAxes, fontsize=12, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

    plt.tight_layout()
    plt.savefig('results/quality_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Saved: results/quality_heatmap.png")

# Generate quality heatmap
plot_quality_heatmap(model, val_images, val_masks, grid_size=(10, 10))

# ============================================================================
# 4. BEST AND WORST PREDICTIONS
# ============================================================================

def visualize_best_worst(model, image_paths, mask_paths, n_each=5):
    """
    OUTPUT: Side-by-side comparison of best and worst predictions
    PURPOSE: Understand where model excels and struggles
    """
    print(f"\nAnalyzing best and worst predictions ({n_each} each)...")

    predictions = []

    for i in range(min(100, len(image_paths))):
        img = cv2.imread(image_paths[i])
        img_resized = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

        mask = cv2.imread(mask_paths[i], cv2.IMREAD_GRAYSCALE)
        mask_resized = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))

        img_input = img_resized.astype(np.float32) / 255.0
        img_input = np.expand_dims(img_input, axis=0)
        pred = model.predict(img_input, verbose=0)[0].squeeze()
        pred_binary = (pred > 0.5).astype(np.uint8)

        mask_binary = (mask_resized > 127).astype(np.uint8)

        intersection = np.sum(mask_binary * pred_binary)
        union = np.sum(mask_binary) + np.sum(pred_binary) - intersection
        iou = intersection / (union + 1e-6)

        predictions.append({'index': i, 'iou': iou})

    predictions_df = pd.DataFrame(predictions)
    predictions_df = predictions_df.sort_values('iou')

    worst_indices = predictions_df.head(n_each)['index'].values
    best_indices = predictions_df.tail(n_each)['index'].values

    fig, axes = plt.subplots(n_each, 8, figsize=(24, 3*n_each))

    for i, idx in enumerate(worst_indices):
        img = cv2.imread(image_paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

        mask = cv2.imread(mask_paths[idx], cv2.IMREAD_GRAYSCALE)
        mask_resized = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))

        img_input = img_resized.astype(np.float32) / 255.0
        img_input = np.expand_dims(img_input, axis=0)
        pred_prob = model.predict(img_input, verbose=0)[0].squeeze()
        pred_binary = (pred_prob > 0.5).astype(np.uint8) * 255

        iou = predictions_df[predictions_df['index'] == idx]['iou'].values[0]

        # Worst predictions (left half)
        axes[i, 0].imshow(img_resized)
        axes[i, 0].set_title(f'Worst #{i+1}\nIoU: {iou:.3f}',
                            fontsize=9, fontweight='bold', color='red')
        axes[i, 0].axis('off')

        axes[i, 1].imshow(mask_resized, cmap='gray')
        axes[i, 1].set_title('GT', fontsize=9)
        axes[i, 1].axis('off')

        axes[i, 2].imshow(pred_prob, cmap='jet')
        axes[i, 2].set_title('Confidence', fontsize=9)
        axes[i, 2].axis('off')

        axes[i, 3].imshow(pred_binary, cmap='gray')
        axes[i, 3].set_title('Prediction', fontsize=9)
        axes[i, 3].axis('off')

    for i, idx in enumerate(best_indices):
        img = cv2.imread(image_paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

        mask = cv2.imread(mask_paths[idx], cv2.IMREAD_GRAYSCALE)
        mask_resized = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))

        img_input = img_resized.astype(np.float32) / 255.0
        img_input = np.expand_dims(img_input, axis=0)
        pred_prob = model.predict(img_input, verbose=0)[0].squeeze()
        pred_binary = (pred_prob > 0.5).astype(np.uint8) * 255

        iou = predictions_df[predictions_df['index'] == idx]['iou'].values[0]

        # Best predictions (right half)
        axes[i, 4].imshow(img_resized)
        axes[i, 4].set_title(f'Best #{i+1}\nIoU: {iou:.3f}',
                            fontsize=9, fontweight='bold', color='green')
        axes[i, 4].axis('off')

        axes[i, 5].imshow(mask_resized, cmap='gray')
        axes[i, 5].set_title('GT', fontsize=9)
        axes[i, 5].axis('off')

        axes[i, 6].imshow(pred_prob, cmap='jet')
        axes[i, 6].set_title('Confidence', fontsize=9)
        axes[i, 6].axis('off')

        axes[i, 7].imshow(pred_binary, cmap='gray')
        axes[i, 7].set_title('Prediction', fontsize=9)
        axes[i, 7].axis('off')

    plt.suptitle('Best vs Worst Predictions', fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig('results/best_worst_predictions.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Saved: results/best_worst_predictions.png")

# Generate best/worst comparison
visualize_best_worst(model, val_images, val_masks, n_each=5)

print("\n" + "="*70)
print("ALL VISUALIZATIONS COMPLETE!")
print("="*70)
print("\nGenerated Files:")
print("  1. results/predictions_with_overlays.png")
print("  2. results/confusion_matrix_validation.png")
print("  3. results/quality_heatmap.png")
print("  4. results/best_worst_predictions.png")
print("\nYou now have:")
print("  ✓ Prediction overlays (red = detected oil spill)")
print("  ✓ Confidence heatmaps showing model certainty")
print("  ✓ Confusion matrix with pixel-level metrics")
print("  ✓ Quality heatmap across 100 samples")
print("  ✓ Best/worst case analysis")
print("="*70)